# Comics Rx
## [A comic book recommendation system](https://github.com/MangrobanGit/comics_rx)
<img src="https://images.unsplash.com/photo-1514329926535-7f6dbfbfb114?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=2850&q=80" width="400" align='left'>

---

# 4 - ALS Model - Reduced Data

This time, as explored in the EDA NB, let's consider removing customers who we feel have too few or too many purchases to influence the model in the intended way.

Examples:
- Too few - Customers who have only bought 1 comic (series).
- Too many - Customers with > 1000 series (for example, think all eBay customers are rolled into one account number).

# Libraries

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2  # 1 would be where you need to specify the files
#%aimport data_fcns

import pandas as pd # dataframes
import os

# Data storage
from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

# import necessary libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.sql.types import (StructType, StructField, IntegerType
#                                ,FloatType, LongType, StringType)
from pyspark.sql.types import *

import pyspark.sql.functions as F
from pyspark.sql.functions import col, explode, lit, isnan, when, count
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Custom
import data_fcns as dfc
import keys  # Custom keys lib
import comic_recs as cr

# Data storage
from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

import time

import numpy as np

In [2]:
# instantiate SparkSession object
spark = pyspark.sql.SparkSession.builder.master("local[*]").getOrCreate()
# spark = SparkSession.builder.master("local").getOrCreate()

## Import Data

We've previously set aside the dataset into a `json` file.

In [3]:
# We have previously created a version of the transactions table and filtered it down.
sold = spark.read.json('raw_data/als_input_filtered.json')

In [4]:
# Persist the data
sold.persist()

DataFrame[account_id: bigint, bought: bigint, comic_id: bigint]

### ALS Model

Let's start with  train/test split.

In [5]:
# Split data into training and test set
(train, test) = sold.randomSplit([.8, .2], seed=41916)

Make sure shapes make sense.

In [6]:
print(train.count(), len(train.columns))

44197 3


In [7]:
print(test.count(), len(test.columns))

11145 3


In [29]:
# Evaluate the model by computing the RMSE on the test data
eval_reg = RegressionEvaluator(metricName="rmse"
                               , labelCol="bought"
                               , predictionCol="prediction")

In [48]:
rmse = eval_reg.evaluate(predictions)

In [38]:
# hyper-param config
num_iterations = 20
ranks = [10]
reg_params = [0.1]
alphas = [40]
# reg_params = [0.001, 0.2]

In [39]:
# grid search and select best model
start_time = time.time()
final_model = train_ALS(train, test, num_iterations, reg_params, ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

Py4JJavaError: An error occurred while calling o458.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.lang.StackOverflowError
java.lang.StackOverflowError
	at java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1534)
	at java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1509)
	at java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1432)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1178)
	at java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1548)
	at java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1509)
	at java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1432)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1178)
	at java.io.ObjectOutputStream.writeObject(ObjectOutputStream.java:348)
	at scala.collection.immutable.List$SerializationProxy.writeObject(List.scala:479)
	at sun.re

In [37]:
def train_ALS(train, test, num_iters, reg_param, ranks, alphas):
    """
    Grid Search Function to select the best model based on RMSE of hold-out data
    """
    # initial
    min_rmse = float('inf')
    best_rank = -1
    best_regularization = 0
    best_alpha = 1
    best_model = None
    
    for rank in ranks:
        for reg in reg_param:
            for alpha in alphas:
                # train ALS model
#                 model = ALS(
#                     ratings=train_data,    # (userID, productID, rating) tuple
#                     iterations=num_iters,
#                     rank=rank,
#                     lambda_=reg,           # regularization param
#                     seed=99)

                als = ALS(maxIter=num_iters,
                      rank=rank,
                      userCol='account_id',
                      itemCol='comic_id',
                      ratingCol='bought',
                      implicitPrefs=True,
                      regParam=reg,
                      alpha=alpha,
                      coldStartStrategy='drop', #Just for CV
                      seed=41916)

                model = als.fit(train)

                # Generate predictions on Test
                predictions = model.transform(test)
                predictions.persist()
                
                rmse = eval_reg.evaluate(predictions)
                
                print('{} latent factors and regularization = {}: validation RMSE is {}'.format(rank, reg, rmse))
               
                if rmse < min_rmse:
                    best_rank = rank
                    best_regularization = reg
                    best_alpha = alpha
                    best_model = model

    return best_model

                    #     print('\nThe best model has {} latent factors and regularization = {}'.format(best_rank, best_regularization))
    
    #                 valid_data = validation_data.map(lambda p: (p[0], p[1]))
#                 predictions = model.predictAll(valid_data).map(lambda r: ((r[0], r[1]), r[2]))
                
#                 # get the rating result
#                 ratesAndPreds = validation_data.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
                
#                 # get the RMSE
#                 MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
#                 error = math.sqrt(MSE)
                
#                 print('{} latent factors and regularization = {}: validation RMSE is {}'.format(rank, reg, error))
                
#                 if error < min_error:
#                     min_error = error
#                     best_rank = rank
#                     best_regularization = reg
#                best_model = model


In [41]:
# Create ALS instance and fit model
als = ALS(maxIter=20,
          rank=10,
          userCol='account_id',
          itemCol='comic_id',
          ratingCol='bought',
          implicitPrefs=True,
          alpha=40,
          seed=41916)
model = als.fit(train)

In [42]:
now = time.ctime(int(time.time()))
print("Completed on {}.".format(now))

Completed on Fri Jun 28 10:52:49 2019.


### Evaluation on Test

In [43]:
# Generate predictions on TEST
predictions = model.transform(test)
predictions.persist()

DataFrame[account_id: int, comic_id: bigint, bought: int, prediction: float]

In [44]:
predictions.show(10)

+----------+--------+------+-----------+
|account_id|comic_id|bought| prediction|
+----------+--------+------+-----------+
|      1842|     471|     1|   0.323991|
|      1489|     496|     1|-0.13128927|
|       690|     496|     1|0.062074617|
|      3006|     833|     1| 0.25017345|
|      1089|     833|     1| 0.36356872|
|      2777|     833|     1|  0.3026107|
|      2770|     833|     1| 0.16230237|
|      2408|     833|     1| 0.48116648|
|      2123|     833|     1| 0.64031804|
|      1645|    1342|     1| 0.08538671|
+----------+--------+------+-----------+
only showing top 10 rows



`BinaryClassificationEvaluator` only likes doubles for `rawPredictionCol`, so cast it.

In [45]:
predictions = predictions.withColumn("prediction", predictions["prediction"].cast(DoubleType()))

In [46]:
predictions.show(10)

+----------+--------+------+-------------------+
|account_id|comic_id|bought|         prediction|
+----------+--------+------+-------------------+
|      1842|     471|     1|0.32399100065231323|
|      1489|     496|     1|-0.1312892735004425|
|       690|     496|     1|0.06207461655139923|
|      3006|     833|     1| 0.2501734495162964|
|      1089|     833|     1| 0.3635687232017517|
|      2777|     833|     1|0.30261069536209106|
|      2770|     833|     1|0.16230237483978271|
|      2408|     833|     1| 0.4811664819717407|
|      2123|     833|     1| 0.6403180360794067|
|      1645|    1342|     1|0.08538670837879181|
+----------+--------+------+-------------------+
only showing top 10 rows



### Initial Evaluation

Based on our first swing of the bat:
- `maxIter` = 20
- `rank` = 10

In [47]:
# Evaluate the model by computing the RMSE on the test data
eval_reg = RegressionEvaluator(metricName="rmse", labelCol="bought",
                                predictionCol="prediction")

In [48]:
rmse = eval_reg.evaluate(predictions)

In [49]:
print("RMSE= " + str(rmse))

RMSE= nan


Oh boy, better remove some nans (for now).

evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC'
                                          ,labelCol='bought'
                                          ,rawPredictionCol='prediction'
                                          )

auc = evaluator.evaluate(predictions)

print("Area Under the Curve = " + str(auc))

## Check nans

In [50]:
# Convert to pandas dataframe
pred_df = predictions.select('*').toPandas()

# Check nulls
pred_num_nulls = pred_df['prediction'].isna().sum()

# Num rows
preds_attempted = pred_df.shape[0]

In [51]:
print("There are {} nulls out of {}.".format(pred_num_nulls, preds_attempted))

There are 342 nulls out of 11048.


In [52]:
print("So {:.2f}% are nulls.".format(pred_num_nulls/preds_attempted))

So 0.03% are nulls.


Since, they're such a tiny portion of the test set, for now let's remove the nulls for now.

In [53]:
predictions.count()

11048

In [54]:
# Convert back to spark dataframe
predictions = spark.createDataFrame(pred_df)

In [55]:
predictions.select([count(when(isnan(c), c)).alias(c) for c in predictions.columns]).show()

+----------+--------+------+----------+
|account_id|comic_id|bought|prediction|
+----------+--------+------+----------+
|         0|       0|     0|       342|
+----------+--------+------+----------+



In [56]:
pred_no_na = predictions.na.drop()

In [57]:
pred_no_na.persist()

DataFrame[account_id: bigint, comic_id: bigint, bought: bigint, prediction: double]

In [58]:
pred_no_na.count()

10706

In [59]:
# Evaluate the model by computing the rmse on the test data
rmse2 = eval_reg.evaluate(pred_no_na)

print("RMSE = " + str(rmse2))

RMSE = 0.49250295110493914


Ok, so that gives us a baseline. Let's see if we can do a little grid search action.

### Get Top N recommendations for Single User

Let's make a reference list of `account_id`'s, for testing purposes.

In [61]:
n_to_test = 5

users = (tot_sold_ids_only.select(als.getUserCol())
                          .sample(False
                                  ,n_to_test/tot_sold_ids_only.count()
                                  ,41916)
        )
users.persist()
users.show()

+----------+
|account_id|
+----------+
|      2185|
|      1663|
|         5|
|       775|
|       604|
+----------+



We developed and wrote the functionality out to a function in `comic_recs.py`

###  Testing function!

- Pass the function to a pandas dataframe. 
- Function will ask for an account_id.
- Will return top n, n defined in parameters.

In [62]:
top_n_df = cr.get_top_n_recs_for_user(spark=spark, model=model, topn=5)
top_n_df

2185


,comic_title
1,Bitch Planet Triple Feature (Image)
2,Redlands (Image)
3,Death Or Glory (Image)
4,Isola (Image)
5,Unsound (Boom)


In [63]:
top_n_df = cr.get_top_n_recs_for_user(spark=spark, model=model, topn=5)
top_n_df

604


,comic_title
1,Amazing Spider-Man Annual (Marvel)
2,Astonishing X-Men Annual (Marvel)
3,Hulk (Marvel)
4,Deadpool Annual (Marvel)
5,Astonishing X-Men (Marvel)


In [89]:
top_n_df = cr.get_top_n_recs_for_user(spark=spark, model=model, topn=20)
top_n_df

161


,comic_title
1,Doom Patrol (DC)
2,Deadpool (Marvel)
3,Punisher (Marvel)
4,Mighty Thor (Marvel)
5,Beef (Image)
6,Invincible (Image)
7,X-Men Grand Design (Marvel)
8,Moon Knight (Marvel)
9,Reborn (Image)
10,Space Riders Galaxy of Brutal (Other)


## Conclusions after single run
- Seems realistic? Only three tests, but the results seem 'individualized' in the sense that there is no overlap between the sets (albeit small samples).

---

## Grid Search

In [77]:
# Create a parameter grid
params = (ParamGridBuilder()
         # .addGrid(als.regParam, [1, 0.1])
          .addGrid(als.maxIter, [20])
          .addGrid(als.rank, [10])
          .addGrid(als.alpha, [40])).build()

cv = TrainValidationSplit(estimator=als
                          , evaluator=eval_reg
                          , estimatorParamMaps=params
                          , trainRatio=0.8)
#cv = CrossValidator(estimator=als_implicit, estimatorParamMaps=paramGrid, evaluator=RegressionEvaluator())
model_implicit = cv.fit(train)

In [78]:
### Cross validate for best hyperparameters
cv = CrossValidator(estimator=als
                    ,estimatorParamMaps=params
                    ,evaluator=eval_reg
                    ,parallelism=4
                   )

In [79]:
now = time.ctime(int(time.time()))
print("Completed on {}.".format(now))

Completed on Fri Jun 28 14:05:20 2019.


In [80]:
# Fit and store model
best_model = cv.fit(test)

In [81]:
als_model = best_model.bestModel

In [85]:
print(als_model.explainParams())

coldStartStrategy: strategy for dealing with unknown or new users/items at prediction time. This may be useful in cross-validation or production scenarios, for handling user/item ids the model has not seen in the training data. Supported values: nan,drop. (default: nan)
itemCol: column name for item ids. Ids must be within the integer value range. (default: item, current: comic_id)
predictionCol: prediction column name (default: prediction)
userCol: column name for user ids. Ids must be within the integer value range. (default: user, current: account_id)


In [88]:
!ls

LICENSE       archive	       comics_rx-1_data_prep.ipynb	   data_fcns.py
README.md     assets	       comics_rx-2_eda.ipynb		   keys.py
__pycache__   code_archive.py  comics_rx-3_als_all_data.ipynb	   raw_data
als_filtered  comic_recs.py    comics_rx-4_als_reduced_data.ipynb  scratch


In [87]:
als_model.save('als_filtered')

In [82]:
now = time.ctime(int(time.time()))
print("Completed on {}.".format(now))

Completed on Fri Jun 28 14:06:20 2019.


---

## Retrieving Saved Model

In [1]:
comic_rec_model = ALSModel.load('als_filtered')

NameError: name 'ALSModel' is not defined

In [ ]:
top_n_df = cr.get_top_n_recs_for_user(spark=spark, model=comic_rec_model, topn=50)
top_n_df